In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
import pgeocode



/Users/clarity./Desktop/Columbia/FALL 2025/STAT 5025/Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
print("Token loaded:", bool(os.getenv("app_token")))
app_token = os.getenv("app_token")



Token loaded: True


In [3]:
def fetch_nyc_dataset(dataset_id, limit=1000000):
    base_url = f"https://data.cityofnewyork.us/resource/{dataset_id}.json"
    headers = {"app_token": app_token} if app_token else {}
    params = {"$limit": limit}

    response = requests.get(base_url, headers=headers, params=params)
    response.raise_for_status()
    df = pd.DataFrame(response.json())
    print(f"Retrieved {len(df)} rows and {len(df.columns)} columns")
    return df

In [6]:
dataset_ids = {
    'rodent': "p937-wjvj",
    'food_scrap': "if26-z6xq",
    'garage': "xw3j-2yxf",
    'litter_basket': "8znf-7b2c",
    'parks': "enfh-gkve"}

food_scrap_df = fetch_nyc_dataset(dataset_id = dataset_ids.get('food_scrap'))
garage_df = fetch_nyc_dataset(dataset_id = dataset_ids.get('garage'))
litter_basket_df = fetch_nyc_dataset(dataset_id = dataset_ids.get('litter_basket'))
parks_df = fetch_nyc_dataset(dataset_id = dataset_ids.get('parks'))


Retrieved 591 rows and 32 columns
Retrieved 80 rows and 18 columns
Retrieved 80 rows and 18 columns
Retrieved 24681 rows and 19 columns
Retrieved 24681 rows and 19 columns
Retrieved 2054 rows and 34 columns
Retrieved 2054 rows and 34 columns


In [9]:
# ----- RAT INSPECTION PORTAL (RIP) DATA ----- #
dataset_id = "p937-wjvj"
url = f"https://data.cityofnewyork.us/resource/{dataset_id}.json"

headers = {"app_token": app_token}
params = {
    '$limit': 1000000,
    '$where': "inspection_date >= '2023-01-01T00:00:00' AND inspection_date < '2024-01-01T00:00:00' AND inspection_type='Initial'"
}

response = requests.get(url, headers=headers, params=params)
rat_df = pd.DataFrame(response.json())

In [10]:
# ----- POPULATION DATA ----- #
url = "https://api.census.gov/data/2022/acs/acs5"

params = {
    'get': 'NAME,B01003_001E',
    'for': 'zip code tabulation area:*',
    #'key': CENSUS_KEY
}

response = requests.get(url, params=params, timeout=30)
data = response.json()
df = pd.DataFrame(data[1:], columns=data[0])
df.columns = ['name', 'population', 'ZIP']

# Filtering for NYC zip codes
pop_df = df[df['ZIP'].str.startswith(('100', '101', '102', '103', '104', 
    '111', '112', '113', '114', '116'))].reset_index(drop=True)



In [ ]:
regression_df = rat_df.groupby('zip_code').size().reset_index(name='inspections')

regression_df = regression_df.merge(
    pop_df[['ZIP', 'population']],  # Set ZIP as index, adds only the population column
    left_on='zip_code',
    right_on='ZIP',  # Merge on the index instead of a column
    how='inner' # Keep only rows with values for both inspections and population
).drop(columns=['ZIP'])

#regression_df.head()

,zip_code,inspections
0,0,297
1,10000,1
2,10001,182
3,10002,3922
4,10003,2576


In [ ]:
print("Regression DataFrame:")
print(regression_df)

In [17]:
 #Zip codes that have a DSNY garage
zips_with_garages = set(garage_df['zip'].unique())

# Adding a dummy variable: 1 if the zip code has a garage, 0 if not
regression_df['has_garage'] = regression_df['zip_code'].isin(zips_with_garages).astype(int)

# Initialize geocoder and get postal data
geocoder = pgeocode.Nominatim('us')
postal_df = geocoder._data_frame

# Convert to numeric
food_scrap_df['latitude'] = pd.to_numeric(food_scrap_df['latitude'], errors='coerce')
food_scrap_df['longitude'] = pd.to_numeric(food_scrap_df['longitude'], errors='coerce')


In [ ]:
# Function to find nearest ZIP
def get_zip(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return None

    distances = np.sqrt(
        (postal_df['latitude'] - lat)**2 + 
        (postal_df['longitude'] - lon)**2
    )
    nearest_idx = distances.idxmin()
    return str(int(postal_df.loc[nearest_idx, 'postal_code']))

# Apply to DataFrame
food_scrap_df['zip_code'] = food_scrap_df.apply(lambda row: get_zip(row['latitude'], row['longitude']), axis=1)

# Zip codes that have a DSNY garage
zips_with_dropoffs = set(food_scrap_df['zip_code'].unique())

# Adding a dummy variable: 1 if the zip code has a dropoff, 0 if not
regression_df['has_dropoff'] = regression_df['zip_code'].isin(zips_with_dropoffs).astype(int)


print(regression_df.head())
print(len(regression_df))
regression_df.to_csv('regression_df.csv', index=False)


  zip_code  inspections population  has_garage  has_dropoff
0    10001          182      27004           0            1
1    10002         3922      76518           1            1
2    10003         2576      53877           0            1
3    10004           18       4579           0            0
4    10005           27       8801           0            0
182


In [ ]:
parks_df
parks_df = parks_df.loc[:, ['zipcode', 'acres', 'omppropid']]
parks_df['acres'] = parks_df['acres'].astype(float)
parks_df = parks_df.dropna(subset=['zipcode', 'acres'])

parks_df['zipcode'] = parks_df['zipcode'].astype(str).str.split(',')
parks_df = parks_df.explode('zipcode')

# 3. Clean formatting and keep only valid 5-digit ZIPs
parks_df['zipcode'] = parks_df['zipcode'].str.strip()

# len(parks_df)
# parks_df.zipcode.nunique() #183
# parks_df.omppropid.nunique() #2052

parks_df['num_zips'] = parks_df['zipcode'].apply(lambda x: len(str(x).split(',')))
parks_df['num_zips'].value_counts().head()
parks_df.head()


In [128]:
gdf = parks_df.groupby('zipcode')
park_size_df = gdf['acres'].sum().reset_index(name='total_park_acres')
park_size_df.head()


,zipcode,total_park_acres
0,1,1247.912
1,10001,14.561
2,10002,90.192
3,10003,13.712
4,10004,23.624


In [123]:
regression_with_parks = regression_df.merge(
    park_size_df,
    left_on='zip_code',
    right_on='zipcode',
    how='left'
)

regression_with_parks.head()
regression_with_parks.to_csv('regression_with_parks.csv', index=False)